In [ ]:
#Prima cella
# programma per gestione combinata dei tre EVSEs (Delta DC o "Delta", Alfen AC o "Alfen", Autel AC o "Autel")

import time
import threading
import keyboard

#from pymodbus.server import StartSerialServer
#from pymodbus.transaction import ModbusRtuFramer
from pymodbus.datastore import  (ModbusSequentialDataBlock, ModbusSlaveContext, ModbusServerContext)
from pymodbus.constants import Endian
from pymodbus.payload import BinaryPayloadBuilder
from pymodbus.payload import BinaryPayloadDecoder
from pymodbus.client import ModbusTcpClient

#from pyModbusTCP.client import ModbusClient
from datetime import datetime
import struct
import math

#Definizione di tutti gli indirizzi IP usati nel setup sperimentale
IP_Mc = '192.168.170.170' #meter centrale
IP_M_Autel = '192.168.170.171' #meter Autel
IP_M_Alfen = '192.168.170.172' #meter Alfen
IP_M_Delta = '192.168.170.173' #meter Delta
IP_M_home_HH = '192.168.170.174' #meter load emulator HH
IP_M_PV = '192.168.170.175' #meter CINERGIA
IP_Alfen = '192.168.170.120' #EVSE Alfen
IP_Autel = '192.168.170.130' #EVSE Autel
IP_Delta = '192.168.170.150' #EVSE Delta
#IP_MOXA = '192.168.170.180'
#IP_PC = '192.168.170.253'

Mc = ModbusTcpClient(host = IP_Mc, port = 502) #meter centrale
#client1 = ModbusTcpClient(host = IP_M_Autel, port = 502) #meter Autel
M_Autel = ModbusTcpClient(host = IP_M_Autel, port = 502) #meter Autel
M_Alfen = ModbusTcpClient(host = IP_M_Alfen, port = 502) #meter Alfen
M_Delta = ModbusTcpClient(host = IP_M_Delta, port = 502) #meter Delta
M_home_HH = ModbusTcpClient(host = IP_M_home_HH, port = 502) #meter load emulator HH
M_PV = ModbusTcpClient(host = IP_M_PV, port = 502) #meter CINERGIA
Alfen = ModbusTcpClient(host = IP_Alfen, port = 502) #EVSE Alfen
Autel = ModbusTcpClient(host = IP_Autel, port = 502) #EVSE Autel
Delta = ModbusTcpClient(host = IP_Delta, port = 5020) #EVSE Delta

#Definizione POTENZA FORNITURA
P_forn = float(9000) #[W]
#Riserva di carica % per Autel
#Ris = float(5) #per l'Autel pongo la riserva di carica al minimo 5%
#calcolo della p_allocabile per la Autel
#P_allocabile = P_forn-0.05*P_forn-(Ris/100)*P_forn #p_allocabile è quella massima resa disponibile alla Autel
#potenza ricarica max EV AUTEL
#P_ricarica_max = float(6500)  #es: 6,6 kW per Nissan Leaf, 7,4 kW se usi modello EV monofase da CDS

#I_max_Delta_address: int = 258 #indirizzo modbus su cui scrivere setpoint corrente Delta
I_max_Alfen_address: int = 1210 #indirizzo modbus su cui scrivere setpoint corrente Alfen


In [ ]:
#Seconda cella

def stop():
    #stop ricarica per Delta
    while(True):
        #print("Premere ALT per stop Delta")
        if keyboard.is_pressed('alt'):  #se invece stoppare la ricarica
            lock.acquire()
            #Delta_WriteRegister(I_max_Delta_address,6) #scrittura su Delta 6A, in modo che al prossimo lancio di ricarica questo parte da 6A
            #Delta_WriteRegister(257,2) #scrittura su Delta
            Delta_WriteRegister(600,500) #scrittura su Delta. Impone 500W ->dovrebbe staccare la ricarica
            lock.release()
            #Delta_WriteRegister(Delta,I_max_Delta_address,6) #imponi 6A come limite max per prossima ripartenza
            print("Ricarica Delta stoppata")
            time.sleep(5) #per lasciare il tempo di stoppare la ricarica


#METER
#definisco la funzione di lettura meter Siemens del registro 65 potenza attiva totale
# registro 25+26 (potenza attiva su L1)

def SiemensReadPower (siemens):
    if (siemens.connect()):
        reg = siemens.read_holding_registers(65, 2, slave=1) #legge il registro della potenza attiva della wallbox
        decoder = BinaryPayloadDecoder.fromRegisters(reg.registers, Endian.BIG, wordorder=Endian.BIG)
        f = decoder.decode_32bit_float()
        siemens.close()
        f = round(f,1)

    return f 
    
#definisco la funzione di lettura meter Siemens dei registri qualsiasi addr e addr+1 
def SiemensRead (siemens, addr):
    if (siemens.connect()):
        reg = siemens.read_holding_registers(addr, 2, slave=1) #legge il registro della potenza attiva della wallbox
        decoder = BinaryPayloadDecoder.fromRegisters(reg.registers, Endian.BIG, wordorder=Endian.BIG)
        f = decoder.decode_32bit_float()
        siemens.close()
        #return f 
    return f 

#funzione che restituisce la potenza assorbita dalla Delta ottenuta per differenza tra potenza totale(Mc) - potenza Autel - potenza Alfen
def SiemensDelta():
    f = SiemensReadPower(Mc) - SiemensReadPower(M_Alfen) - SiemensReadPower(M_Autel)
    return f



#ALFEN
Alfen_addr_P = 344  #registro potenza attiva Alfen. float32
Alfen_addr_V = 306  #registro tensione fase 1 Alfen. float32

#definisco la funzione di lettura registri su EVSE Alfen (Alfen AC). Registro 1210 (lettura/scrittura) max corrente 
def AlfenRead(addr):
    if (Alfen.connect()):
        reg = Alfen.read_holding_registers(addr, 2, slave=1) #legge il registro addr e addr+1
        decoder = BinaryPayloadDecoder.fromRegisters(reg.registers, Endian.BIG, wordorder=Endian.BIG)
        f = decoder.decode_32bit_float()
        Alfen.close()
    else:
        for maxIter in range(10):
            time.sleep(1)
            print('Tentativi connessione Alfen ',maxIter)
            if(Alfen.connect()):
                reg = Alfen.read_holding_registers(addr, 2, slave=1) #legge il registro addr e addr+1
                decoder = BinaryPayloadDecoder.fromRegisters(reg.registers, Endian.BIG, wordorder=Endian.BIG)
                f = decoder.decode_32bit_float()
                Alfen.close()
    return f

#definisco la funzione di scrittura registri su EVSE Alfen (Alfen AC). Usata per scrivere la max corrente (floating point 32 bit)
#If you need to write floating point values, you'll have to build a payload and then write it to the registers   
#Registro 1210 (lettura/scrittura) max corrente 
def Alfen_WriteRegister(val):
    if (Alfen.connect()):
        builder = BinaryPayloadBuilder(wordorder=Endian.BIG, byteorder=Endian.BIG)    
        builder.add_32bit_float(val)
        payload = builder.to_registers()
        Alfen.write_registers(1210, payload, slave=1)
        Alfen.close()

#Funzione che setta la potenza di ricarica (in kW) desiderata dell'Alfen. 2% margine
def AlfenSetPower(val):
    if (val == 0):
        Alfen_WriteRegister(0)
        print("Stoppo la ricarica dell'Alfen")
        time.sleep(3)
        if SiemensReadPower(M_Alfen) > 100:
            print("! l'Alfen sta assorbendo più di 100W -> ", round(SiemensReadPower(M_Alfen),1), "W")
    else:
        #prima iterazione
        tensione = AlfenRead(Alfen_addr_V)
        corrente = round(0.98*val*1000/tensione, 2)
        Alfen_WriteRegister(corrente)
        time.sleep(4)
        #seconda iterazione
        tensione = AlfenRead(Alfen_addr_V)
        corrente = round(0.98*val*1000/tensione, 2)
        Alfen_WriteRegister(corrente)
        time.sleep(3)

        if SiemensReadPower(M_Alfen) > val*1000:  #se la potenza assorbita è ancora maggiore mette un margine del 5%
            tensione = AlfenRead(Alfen_addr_V)
            corrente = round(0.95*val*1000/tensione, 2)
            Alfen_WriteRegister(corrente)
            time.sleep(3)
            if SiemensReadPower(M_Alfen) > val*1000:
                print("Errore, la potenza dell'Alfen è maggiore del setpoint scelto")
        
        print("L'ALfen sta assorbendo ", round(SiemensReadPower(M_Alfen),1), "W")


#AUTEL
Autel_addr_P = 10021  #registro per la potenza della fase 1. uint32

def AutelRead(addr):  #Funzione lettura input register uint32 slave=1
    if (Autel.connect()):
        reg=Autel.read_input_registers(addr, 2, slave=1)
        decoder = BinaryPayloadDecoder.fromRegisters(reg.registers, Endian.BIG, wordorder=Endian.BIG)
        f = decoder.decode_32bit_uint()
        #Autel.close()
    return f

def AutelSetPower(val):
    if (Autel.connect()):
        val = val*1000
        builder = BinaryPayloadBuilder(wordorder=Endian.BIG, byteorder=Endian.BIG)    
        builder.add_32bit_uint(val)
        payload = builder.to_registers()
        Autel.write_registers(20000, payload, slave=1)
        #Autel.close()



#DELTA
#definisco la funzione di lettura registri su EVSE Delta (Delta DC 50kW). Per leggere la potenza assorbita registro 047 float32
#slave = 2  CCS Combo
#slave = 3  CHAdeMO
#Nel codice va definita la variabile DeltaSlave
def DeltaRead (addr):  #legge float32. Bisogna inserire lo slave (CHAdeMO o CCS Combo)
    f = 0.0
    #lock.acquire()
    if (Delta.connect()):
        reg=Delta.read_input_registers(addr, 2, slave=DeltaSlave)
        #reg = Delta.read_holding_registers(addr, 2, slave=1) #legge il registro addr
        decoder = BinaryPayloadDecoder.fromRegisters(reg.registers, Endian.BIG, wordorder=Endian.BIG)
        f = decoder.decode_32bit_float()
        Delta.close()
    #lock.release()
    return f 

#funzione lettura del setpoint di potenza EVSE Delta. Registro 600 int32 write holding register
def DeltaReadSetpoint ():
    f = 0.0
    #lock.acquire()
    if (Delta.connect()):
        #reg=Delta.read_input_registers(47, 2, slave=2)
        reg = Delta.read_holding_registers(600, 2, slave=DeltaSlave) #legge il registro 600
        decoder = BinaryPayloadDecoder.fromRegisters(reg.registers, Endian.BIG, wordorder=Endian.BIG)
        f = decoder.decode_32bit_int()
        Delta.close()
    #lock.release()
    return f

#definisco la funzione di scrittura registri su EVSE Delta (Delta DC). Per scrivere la max potenza. Registro 600 int32
def DeltaWriteSetpoint(val):
    if (Delta.connect()):
        val = val*1000
        val = int(val) #lo trasforma in int se inserisco un valore float
        builder = BinaryPayloadBuilder(wordorder=Endian.BIG, byteorder=Endian.BIG)    
        builder.add_32bit_int(val)
        payload = builder.to_registers()
        Delta.write_registers(600, payload, slave=DeltaSlave)
        Delta.close()

In [ ]:
#Prova comando Autel
#Autel.connect()  #!!! La Autel carica solo se la connessione modbus è attiva, quindi il pc deve sempre avere la connessione con l'Autel attiva, altrimenti stoppa la ricarica

AutelSetPower(0)
print("Hai 30 secondi per collegare la presa di ricarica")
time.sleep(35)
t=0
while (t<30):

    print("Meter Autel ", SiemensReadPower(M_Autel), "W.  Autel ", AutelRead(Autel_addr_P) )
    time.sleep(3)
    if t==1:
        print("Inizio la ricarica a max potenza")
        AutelSetPower(8)

    if t==15:
        print("Voglio caricare a 3kW")
        print("Modulo la Autel")
        AutelSetPower(3)


    print("t = ", t)
    t=t+1

print("Stoppo la ricarica")
AutelSetPower(0)
time.sleep(5)
print("Meter Autel ", SiemensReadPower(M_Autel), "W.  Autel ", AutelRead(Autel_addr_P) )

Autel.close()

In [ ]:
#Prova comando Alfen
Alfen_WriteRegister(0)
print("Hai 30 secondi per attaccare il cavo di ricarica")
time.sleep(35)

t=0
while (t<30):

    print("Meter Alfen ", SiemensReadPower(M_Alfen), "W.  Alfen ", AlfenRead(addr_P) )
    time.sleep(3)
    if t==1:
        print("Inizio la ricarica a max potenza")
        AlfenSetPower(8)

    if t==15:
        print("Voglio caricare a 3kW")
        print("Modulo la Alfen")
        AlfenSetPower(3)


    print("t = ", t)
    t=t+1

print("Stoppo la ricarica")
AlfenSetPower(0)


In [ ]:
#Prova comando Delta
DeltaSlave = int(input('Premi 2 se stai usando CCS Combo o premi 3 se stai usando CHAdeMO ' ))
#DeltaSlave= 3

DeltaWriteSetpoint(0)
print("Hai 60 secondi per attaccare il cavo di ricarica")

time.sleep(60)

t=0
while (t<70):

    print("La Delta sta erogando ", int(DeltaRead(9)), "W. Il meter segna ", int(SiemensDelta()), "W")
    time.sleep(3)
    if t==1:
        print("Inizio la ricarica a 39kW")
        DeltaWriteSetpoint(39)
        time.sleep(4)

    if t==30:
        setpoint = int(input('Inserire il nuovo setpoint di potenza in kW' ))
        print("Modulo la Delta")
        DeltaWriteSetpoint(setpoint)
        time.sleep(6)
        if SiemensDelta() > setpoint*1000:
            DeltaWriteSetpoint(0.98*setpoint)
            print("Ho corretto la potenza perchè sforava il limite di setpoint")
            time.sleep(6)
            if SiemensDelta() > setpoint*1000:
                print("Non basta il 2% di margine perchè si sfora ancora il setpoint")
                print("Imposto il margine al 5%")
                DeltaWriteSetpoint(0.95*setpoint)
                time.sleep(4)
    
    if t==50:
        setpoint = int(input('Inserire il nuovo setpoint di potenza in kW' ))
        print("Modulo la Delta")
        DeltaWriteSetpoint(setpoint)
        time.sleep(6)
        if SiemensDelta() > setpoint*1000:
            DeltaWriteSetpoint(0.98*setpoint)
            print("Ho corretto la potenza perchè sforava il limite di setpoint")
            time.sleep(6)
            if SiemensDelta() > setpoint*1000:
                print("Non basta il 2% di margine perchè si sfora ancora il setpoint")
                print("Imposto il margine al 5%")
                DeltaWriteSetpoint(0.95*setpoint)
                time.sleep(4)

    #print("t = ", t)
    t=t+1

print("Stoppo la ricarica")
DeltaWriteSetpoint(0)


In [ ]:
#Terza cella

#Output per Delta.  Restituisce potenza, corrente e soc della delta
def Output_Delta():
    power_Delta = SiemensReadPower(M_Delta)       #con lettura Delta potenza attiva
    power_Delta_kW = float(power_Delta/1000)
    currentDelta = SiemensRead(M_Delta, 13)     #con lettura Delta corrente
    #setpointDelta = DeltaRead(I_max_Delta_address) #Delta
    soc = DeltaRead(11) #EV SOC. !Il SOC è UINT16 e non float32
    #print("Delta assorbe", "%.2f" % power_Delta_kW, "kW e", "%.2f" % currentDelta, "A. Setpoint Delta", setpointDelta,"A. SOC", soc,"%")
    print("Delta assorbe", "%.2f" % power_Delta_kW, "kW e", "%.2f" % currentDelta, "A. SOC", soc/10,"%")

#Output per Alfen.  Restituisce potenza e corrente assorbiti
def Output_Alfen():
    #powerAlfen = AlfenRead(344)       #con lettura Alfen potenza attiva
    powerAlfen = SiemensReadPower(M_Alfen)
    power_Alfen_kW = float(powerAlfen/1000)
    
    currentAlfen = SiemensRead(M_Alfen, 13)
       
    #max_currentAlfen = AlfenRead(1210)
    #if (not max_currentAlfen["err"]):
       # print("Alfen assorbe", "%.2f" % power_Alfen_kW, "kW e", "%.2f" % currentAlfen, "A. Setpoint Alfen",  max_currentAlfen["f"], "A")
        
    #else:
        #print ("Error in Alfen_Read")
    print("Alfen assorbe", "%.2f" % power_Alfen_kW, "kW e", "%.2f" % currentAlfen, "A. Setpoint Alfen")

#Output per Autel.  Restituisce potenza assorbita
def Output_Autel(setpoint_Autel):
    powerAutel = SiemensReadPower (M_Autel)
    powerAutel_kW = float(powerAutel/1000)
    #P_desiderata_kW = setpoint_Autel/1000
    #print("Autel assorbe:", "%.2f" % powerAutel_kW, "kW Setpoint Autel", "%.2f" % P_desiderata_kW, "kW" )
    print("Autel assorbe:", "%.2f" % powerAutel_kW, "kW")


In [ ]:

def meter():
    # INDIRIZZI IP DEI METER
#(al PC server assegnato IP 192.168.170.169)

    IP_Meter_Centrale = '192.168.170.170'       #meter centrale
    IP_Meter_Autel= '192.168.170.171'       #meter della colonnina Autel AC  
    IP_Meter_Alfen = '192.168.170.172'       #meter della colonnina Alfen AC
    IP_Meter_Delta = '192.168.170.173'       #meter della colonnina Delta DC
    IP_Meter_HH = '192.168.170.174'       #meter del carico domestico HH
    IP_Meter_PV = '192.168.170.175'       #meter del PV cinergia   


    timeout_meter = 1
    port = 502
    granularity=0.5 #seconds
# DEFINIZIONE DEI METER: definisco i client che interrogano i meter implementati
#

# Mc=ModbusClient(host= IP_Meter_Centrale, port= 502)
    M_C = ModbusClient(host = IP_Meter_Centrale, auto_open=True, auto_close=True, debug=False, timeout=timeout_meter) 
    M_Autel = ModbusClient(host = IP_Meter_Autel, auto_open=True, auto_close=True, debug=False, timeout=timeout_meter,port=port)
    M_Alfen = ModbusClient(host = IP_Meter_Alfen, auto_open=True, auto_close=True, debug=False, timeout=timeout_meter,port=port)
    M_Delta = ModbusClient(host = IP_Meter_Delta, auto_open=True, auto_close=True, debug=False, timeout=timeout_meter,port=port)
    M_HH = ModbusClient(host = IP_Meter_HH, auto_open=True, auto_close=True, debug=False, timeout=timeout_meter,port=port)
    M_PV = ModbusClient(host = IP_Meter_PV, auto_open=True, auto_close=True, debug=False, timeout=timeout_meter,port=port)


    while True:
     #client.connect()
     
        try: 
            rr = M_C.read_holding_registers(1,36) 
            test=[struct.unpack('!f', struct.pack('!I', (int(rr[idx]) << 16) | int(rr[idx+1])))[0] for idx in range(0,len(rr),2)]
        except TypeError:
        #test=[]
            print('error')
    
        try:
            rr2=M_Autel.read_holding_registers(1,36)
            test2=[struct.unpack('!f', struct.pack('!I', (int(rr2[idx]) << 16) | int(rr2[idx+1])))[0] for idx in range(0,len(rr2),2)]
        except TypeError:
            print('error')

        try:
            rr3=M_Alfen.read_holding_registers(1,36)
            test3=[struct.unpack('!f', struct.pack('!I', (int(rr3[idx]) << 16) | int(rr3[idx+1])))[0] for idx in range(0,len(rr3),2)]
        except TypeError:
            print('error')

        try:
            rr4=M_Delta.read_holding_registers(1,36)
            test4=[struct.unpack('!f', struct.pack('!I', (int(rr4[idx]) << 16) | int(rr4[idx+1])))[0] for idx in range(0,len(rr4),2)]
        except TypeError:
            print('error')

        try:
            rr5=M_HH.read_holding_registers(1,36)
            test5=[struct.unpack('!f', struct.pack('!I', (int(rr5[idx]) << 16) | int(rr5[idx+1])))[0] for idx in range(0,len(rr5),2)]
        except TypeError:
            print('error')

        try:
            rr6=M_PV.read_holding_registers(1,36)
            test6=[struct.unpack('!f', struct.pack('!I', (int(rr6[idx]) << 16) | int(rr6[idx+1])))[0] for idx in range(0,len(rr6),2)]
        except TypeError:
            print('error')

        with open('M_C.csv','a') as file:
            file.write(datetime.now().isoformat()+','+str(test)[1:-1]+','+'meter Mc'+','+'\n')
    
    
        with open('M_Autel.csv','a') as file:
            file.write(datetime.now().isoformat()+','+str(test2)[1:-1]+','+'meter Autel'+','+'\n')  

        
        with open('M_Alfen.csv','a') as file:
            file.write(datetime.now().isoformat()+','+str(test3)[1:-1]+','+'meter Alfen'+','+'\n')


        with open('M_Delta.csv','a') as file:
            file.write(datetime.now().isoformat()+','+str(test4)[1:-1]+','+'meter Delta'+','+'\n')

        
        with open('M_HH.csv','a') as file:
            file.write(datetime.now().isoformat()+','+str(test5)[1:-1]+','+'meter HH'+','+'\n')
        

        with open('M_PV.csv','a') as file:
            file.write(datetime.now().isoformat()+','+str(test6)[1:-1]+','+'meter PV'+','+'\n')
        time.sleep(granularity)
    
        if keyboard.is_pressed('esc'):
            print('exit')
            break   



In [ ]:
if __name__ == "__main__":
    thread3 = threading.Thread(target = meter)
    thread4 = threading.Thread(target = stop)
    thread3.start()
    thread4.start()
    thread1 = threading.Thread(target = server)
    #thread1.daemon = True
    thread1.start()
    thread2 = threading.Thread(target = application)
    thread2.start()
    thread2.join()    
    thread1.join()
    thread3.join()
    thread4.join()

